In [34]:
import numpy as np
import math
from copy import copy

# Dynamics Functions
def dynam(state_curr, dt):
    '''
    constant velocity model 
    '''
    state_next = np.zeros_like(state_curr)
    state_next[0] = state_curr[0] + state_curr[2] * dt
    state_next[1] = state_curr[1] + state_curr[3] * dt
    state_next[2:] = state_curr[2:]
    return state_next

def dynamJac(nState, dt):
    F = np.eye(nState)
    F[0, 2] = dt
    F[1, 3] = dt
    return F

# Measurement Functions
def uwb_meas(state_curr):
    return np.linalg.norm(state_curr[0:2])

def uwb_measJac(nState, state_curr):
    H_uwb = np.zeros((1, nState))
    range =  np.linalg.norm(state_curr[0:2])
    H_uwb[0, 0] = state_curr[0] / range
    H_uwb[0, 1] = state_curr[1] / range
    return H_uwb

def gps_measJac(nState, nMeas, G, A):
    H_gps = np.zeros((nMeas, nState))
    H_gps[:, 0:2] = G
    H_gps[:, 4:] = A
    return H_gps


# EKF Equations
def ekf_predict(state_curr, Sig_curr, dt, Q):
    F = dynamJac(len(state_curr), dt)
    state_next = dynam(state_curr, dt)
    Sig_next = F @ Sig_curr @ (F.T) + Q
    return state_next, Sig_next

def ekf_update(state_curr, Sig_curr, y, y_hat, H, R):
    K = Sig_curr @ (H.T @ np.linalg.inv((H @ Sig_curr @ H.T + R)))
    state_next = state_curr + K @ (y - y_hat)
    Sig_next = Sig_curr - K @ H @ Sig_curr
    return state_next, Sig_next

def ekf_gnss_uwb(state_curr, Sig_curr, y, dt, G, A, Q, R, svID2ind, svID):
    '''
    state_curr : [r1 r2 v1 v2 n1 ... nK], state vector length kmax (max number of satellites visible during segment)
    Sig_curr   : Covariance [4 + kmax, 4 + kmax]
    y          : measurement vector, length 2k + w (k - num. of satellites visible at this iter., w - # UWB ranges)
    Q          : process noise [4 + kmax, 4 + kmax]
    R          : measurement noise [2k + w, 2k + w]
    svID2ind   : map between SV IDs and state vector indices
    svID       : list of SVs visible at iteration
    '''

    #- predict -----------------------
    state_t1, Sig_t1 = ekf_predict(state_curr, Sig_curr, dt, Q)

    #- update -----------------------
    nState_mini = 4 + (G.shape[0] // 2)
    nMeas = len(y)

    # extract relevant states corresponding to svs where measurements were recieved
    if nState_mini != len(state_curr):
        state_mini = np.zeros((nState_mini, 1))
        state_mini[0:4] = state_t1[0:4] 
        sind = 4
        for ii in range(k):                     # TODO: this is assuming that reference is always the last one, which is currently what the main loop is using
            jj = svID2ind[svID[ii]]             
            state_mini[sind] = state_t1[jj]
            sind += 1
    else:
        state_mini = copy(state_t1)

    # measurement jacobian [2k + w, 4 + k]
    H_uwb = uwb_measJac(nState_mini, state_mini)
    H_gps = gps_measJac(nState_mini, nMeas - 1, G, A)
    H = np.concatenate((H_gps, H_uwb))
    
    meas_pred = H @ state_mini
    meas_pred[-1] = uwb_meas(state_mini)
    
    state_t2, Sig_t2 = ekf_update(state_mini, Sig_t1, y, meas_pred, H, R)

    if nState_mini != len(state_curr):
        state_next = np.zeros_like(state_curr)
        state_next[0:4] = state_t2[0:4]
        sind = 4
        for ii in range(k):
            jj = svID2ind[svID[ii]]
            state_next[jj] = state_t2[ii]
            sind += 1                       # TODO: how to deal with the covariance 
    else:
        state_next = copy(state_t2)
        Sig_next = copy(Sig_t2)
        
    return state_next, Sig_next

# OTHER FUNCTIONS
def id_map(sv_lst, dim=2):
    id2ind = {}
    state_ind = 2 * dim
    for i in range(len(sv_lst)):
        id2ind[sv_lst[i]] = state_ind
        state_ind += 1
        
    return id2ind



In [2]:
from operator import le
import os
from textwrap import wrap
import numpy as np
import math

from scipy.sparse.linalg.dsolve.linsolve import use_solver
from preprocessing import *
import matplotlib.pyplot as plt
import pandas as pd
from utils import *
import LAMBDA


np.random.seed(0)
cwd = os.getcwd()
dir_path = cwd[:-3]

###################### SETUP ######################

# define the measurement date in YYYY-MM-DD
date = np.datetime64("2021-08-26")

# params
truth_frequency = 4
obs_freq = 2
leap_seconds = 18
x0 = np.array([-2.7011e+06,-4.292e+06, 3.8554e+06])

# observation and grouth truth files
name_obs1 = dir_path + 'data/20210826_data/leader2.21O'
name_obs2 = dir_path + 'data/20210826_data/follower.21O' 
name_eph =  dir_path + 'data/20210826_data/leader2.21N'

ground_truth = np.loadtxt(dir_path + 'data/20210826_data/Ground_Truth_Static.csv', delimiter = ",")  # ENU
uwb_data = np.loadtxt(dir_path + 'data/20210826_data/UWB_Baseline_Static.csv', delimiter = ",")    

# preprocess
traj1, traj2, eph = loadTrajectories(name_obs1, name_obs2, name_eph)
print('trajectories loaded')

t_gps, svs, code1, code2, carrier1, carrier2, cnos, ts = constructMeasurements(traj1, traj2, date, sort_cn0 = False)
cnos = np.array(cnos)
print('measurements constructed')



trajectories loaded
measurements constructed


<ipython-input-2-6ab660398744>:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cnos = np.array(cnos)


In [48]:
################### INITIALIZE FILTER ####################
# max_svs = list(max(svs, key = lambda i: len(i)))  # longest observed list of svs
# kmax = len(max_svs)                                  # max num of svs in trajectory

# TODO: Dummy for debugging                                         
max_svs = ['G08', 'G10', 'G18', 'G21', 'G23', 'G27', 'G32']
kmax = len(max_svs) - 1
# print(max_svs)
# print('kmax', kmax)

id2ind = id_map(max_svs)
w = 1                                             # num UWB range
nState = 4 + kmax
dt = 0.1

x_pre = np.zeros((nState, 1))
x_pre[0] = 19
x_pre[1] = 23
x_pre[2] = 1
x_pre[3] = 5
x_pre[4:] = np.ones((nState - 4, 1))

P_pre = np.eye(nState)
Q = 0.05 * np.eye(nState)

############### other params #######################
ksnr = 200
phase_ratio = 300 	    # sigma phase = sigma code / phase_ratio
gt_xy_init = [19, 23]   # first ground truth x, y coord


f = 1575.42 * 10 ** 6
c = 299792458
lda = c / f

check_num_sats = []


sigma_code = [1 for i in range(8)]
sigma_phase= [0.01 for i in range(8)]
init_n = None
print('entering loop')


#---- match the start time between ground truth and obs. files -------
obs_start_ind = get_obs_startInd(ts, date, ground_truth)
if truth_frequency > obs_freq:
    gt_inds = np.arange(0, len(ground_truth), truth_frequency / obs_freq)
    obs_inds = np.arange(obs_start_ind, obs_start_ind + math.ceil(len(ground_truth) / (truth_frequency / obs_freq)), 1)
else:
    gt_inds = np.arange(0, len(ground_truth), 1)
    obs_inds = np.arange(obs_start_ind, obs_start_ind + (len(ground_truth)-1) * (obs_freq / truth_frequency), obs_freq / truth_frequency)

gt_inds = gt_inds.astype(int)
obs_inds = obs_inds.astype(int)
starting = 1

truth = []
est = []

# for i in range(0, 30):
for i in range(3, 8):               # DUMMY SEG that has all the same satellites
    # get appropriate indices
    gt_ind = gt_inds[i]
    obs_ind = obs_inds[i]
    
    # GROUND TRUTH
    truth_term = ground_truth[gt_ind, 1:]
    truth.append(truth_term[0:2])

    # MEASUREMENTS
    # gps
    sigma_code, sigma_phase = sigmaFromCN0(cnos[obs_ind], ksnr, phase_ratio)
    psi, G, A, sigma = prepareData(t_gps[obs_ind], svs[obs_ind], code1[obs_ind], code2[obs_ind], carrier1[obs_ind], carrier2[obs_ind], eph, plane=False, ref= -1, x0=x0, f=1575.42*10**6, phase_error=0.025)
    svs_obs = svs[obs_ind]              # satellite IDs visible this time step -- including the chosen reference 
    # print(svs_obs)
    # print('psi',psi.shape)
    # print('G',G.shape)
    # print ('A',A.shape)
    # print('sv', len(svs_obs))

    k = psi.shape[0] // 2               # number of satellites visibe this time step
    H = np.zeros((2 * k, G.shape[1]))
    H[:k] = G
    H[k:] = G
    psi -= truth_term[2] * H[:, 2]
    H = H[:, :2]
    # print('H', H.shape)
    # print('k', k)

    uwb_range = uwb_data[gt_ind, 1]    # TODO: check that the ind 1 is the range meas.
    y = np.reshape(np.append(psi, uwb_range), (2 * k + w, 1))

    # EKF
    print('entering filtering')
    R = 0.1 * np.eye(2 * k + w)          # TODO: adjust this dummy variable 
    x_next, P_next = ekf_gnss_uwb(x_pre, P_pre, y, dt, H, A, Q, R, id2ind, svs_obs)

    # LAMBDA
    print('performing lambda')
    C = A
    Qi = np.linalg.inv(sigma)
    Qhat = np.linalg.inv(np.dot(C.T, np.dot(Qi, C)))
    Qahat = (Qhat + Qhat.T) /2
    afixed, sqnorm, Ps, Qzhat, Z, nfixed, mu = LAMBDA.main(x_next[4:], Qahat, 1)
    if afixed.ndim > 1:
        afixed = afixed[:,0]

    # fix position 
    a = psi[:k]
    b =  lda * afixed
    x_fixed =  a - b
    x_fixed = np.dot(np.linalg.pinv(H[:k]), x_fixed)
    x_fixed = np.reshape(x_fixed, (2, 1))
    print('fixed pos', len(x_fixed))

    x_next[0:2] = x_fixed
    est.append(np.ndarray.flatten(x_next[0:2]))

    x_pre = copy(x_next)
    P_pre = copy(P_next)



print ('code done')
print ('truth', truth)
print('est', est)




entering loop
entering filtering
mini = normal
performing lambda
fixed pos 2
entering filtering
mini = normal
performing lambda
fixed pos 2
entering filtering
mini = normal
performing lambda
fixed pos 2
entering filtering
mini = normal
performing lambda
fixed pos 2
entering filtering
mini = normal
performing lambda
fixed pos 2
code done
truth [array([18.912, 23.575]), array([18.912, 23.575]), array([19.204, 23.929]), array([19.204, 23.929]), array([19.307, 23.963])]
est [array([9.83492874, 7.5982555 ]), array([10.1269109 ,  7.76766086]), array([10.05783208,  7.68726401]), array([9.86577363, 7.53784057]), array([9.75376083, 7.49570845])]
